In [12]:
using HomotopyContinuation, HomologyInferenceWithWeakFeatureSize, DynamicPolynomials
hwfs = HomologyInferenceWithWeakFeatureSize
threshold = 1e-7
# Order of bottlenecks
k = 2

2

In [13]:
num_vars = 3
@polyvar q[1:num_vars]
x = q[1]
y = q[2]
z = q[3]
F = [x^2 + y^2 + z^2/2 - 1]
all_points = hwfs.find_points_on_bottleneck_correspondence(F,q,k)
singular_points = all_points["singular"]
nonsingular_points = all_points["nonsingular"]

nondegenerate_nonsingular = [point for point in nonsingular_points if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)]
nondegenerate_singular = [point for point in singular_points if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)]

real_nonsingular = [point for point in nondegenerate_nonsingular if !(abs(imag(hwfs.apply_distance_squared_to_solution(point))) > threshold) ]
putative_real_nonsingular_contributors = [point for point in real_nonsingular if real(hwfs.apply_distance_squared_to_solution(point)) > threshold]
real_singular = [point for point in nondegenerate_singular if !(abs(imag(hwfs.apply_distance_squared_to_solution(point))) > threshold) ]
real_singular_contributors = [point for point in real_singular if real(hwfs.apply_distance_squared_to_solution(point)) > threshold]


distance_system_for_filtering,parameters_for_filtering = hwfs.minimum_distance_start_system(F,q)
start = randn(ComplexF64,num_vars+length(F))
initial_system = subs(distance_system_for_filtering,parameters_for_filtering => start)
initial_solution = solutions(solve(initial_system,start_system = :polyhedral,parameters=:conservative))
distances_nonsingular = [Inf]
if length(putative_real_nonsingular_contributors) > 0
    distances_nonsingular = [hwfs.filter_solution_to_bottleneck(solution,length(q),k,distance_system_for_filtering,start,parameters_for_filtering,initial_solution;threshold=threshold) for solution in putative_real_nonsingular_contributors]
    distances_nonsingular = [dist for dist in distances_nonsingular if dist!=false]
    if length(distances_nonsingular) == 0
        distances_nonsingular = [Inf]
    end
end
distances_singular = [Inf]
if length(distances_singular) > 0 
    distances_singular = [real(sqrt(hwfs.apply_distance_squared_to_solution(point))) for point in real_singular_contributors]
end


4-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0

In [14]:
# This number is the total number of solutions computed on B_k(F)
length(singular_points)+length(nonsingular_points)

67

In [15]:
# This counts the solutions which are contained in \Gamma_k(F)
length(singular_points) - length(nondegenerate_singular) + length(nonsingular_points) - length(nondegenerate_nonsingular)

61

In [16]:
# This counts the solutions which are not contained in \Gamma_k(F)
length(nondegenerate_singular)+length(nondegenerate_nonsingular)

6

In [17]:
# The number of non-isolated solutions contributing 
# to the weak feature size
length(real_singular_contributors)

4

In [18]:
# We can inspect the points in real_singular_contributors 
# and observe that they all lie on the component defined by 
# { ((x,y,0),(-x,-y,0),1/2,1/2) | (x,y,0) \in V(G) }
println("Circumcenters")
for point in real_singular_contributors
    println(point["circumcenter"])
end
println("")
println("Multipliers for affine combination")
for point in real_singular_contributors
    println(point["multipliers"][1],", ",point["multipliers"][2])
end
println("")
println("Endpoints")
i = 1
for point in real_singular_contributors
    println("Solution "*string(i))
    println(point["endpoints"][1])
    println(point["endpoints"][2])
    i += 1
end
    

Circumcenters
Complex{Float64}[-2.7755575615628914e-16 - 1.6653345369377348e-16im, 2.220446049250313e-16 - 1.1102230246251565e-16im, -6.437450399132673e-20 - 9.9272261418204e-19im]
Complex{Float64}[-3.3306690738754696e-16 + 3.3306690738754696e-16im, 3.3306690738754696e-16 + 2.7755575615628914e-16im, 8.300922883092143e-20 + 2.7803856493622413e-19im]
Complex{Float64}[-2.7755575615628914e-17 + 3.469446951953614e-18im, -5.551115123125783e-17 + 8.673617379884035e-18im, 1.8634724839594607e-20 + 8.682087709356578e-21im]
Complex{Float64}[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 9.80908925027372e-45im]

Multipliers for affine combination
0.5 + 1.530842645572697e-17im, 0.5000000000000001 - 1.5309273488674224e-17im
0.5 + 2.999089556343301e-17im, 0.5000000000000001 - 3.001207138711437e-17im
0.5 + 2.0438905017246267e-18im, 0.5 - 2.0396553369883552e-18im
0.5 - 6.637715322642147e-34im, 0.5 + 3.633846094315247e-33im

Endpoints
Solution 1
Complex{Float64}[0.8642009281583206 + 0.7303990216772966im, -1.0663234800

In [19]:
# All non-degenerate non-singular with real distance
println(length(putative_real_nonsingular_contributors)/factorial(k))
# Real algebraic bottlenecks of V(F)
println(length([point for point in putative_real_nonsingular_contributors if hwfs.check_if_vector_is_real(point["circumcenter"];threshold=threshold)])/factorial(k))
# Real algebraic bottlenecks of V(F) \cap R^3
println(length([point for point in putative_real_nonsingular_contributors if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)])/factorial(k))

1.0
1.0
1.0


In [20]:
# Critical values from non-isolated contributors
minimum(distances_singular)

1.0

In [21]:
# Critical values from isolated geometric k-bottlenecks
minimum(distances_nonsingular)

Inf

In [22]:
# Example: This function performs all necessary
# computations and finds the weak feature size 
compute_weak_feature_size(F;maximum_bottleneck_order=2)

1.0